In [ ]:
import cv2 
import numpy as np
import glob
import os
import shutil

import matplotlib.pyplot as plt

import bat_functions as kbf
# from bat_functions import process_frame, get_tracked_bats_in_frame, initialize_image_array

In [ ]:
def generate_annotation_frame(info_dict):
    """ Generate mask for image and save both.
    
    Args:
        info_dict: dictionary that contains:
            frame_files: list of frame files in clip
            focal_frame: frame ind to process
            save_folder: Path to folder to save images and masks
            save_images: boolean should image and mask be saved
    """
    frame_files = info_dict['frame_files']
    focal_frame = info_dict['focal_frame']
    save_folder = info_dict['save_folder']
    save_images = info_dict['save_images']
    return_result = info_dict['return_result']
    
    # index of focal_frame in images array
    focal_frame_ind = 15
    array_size = 31
    
    images = []
    
    for file in frame_files[focal_frame-focal_frame_ind:array_size-focal_frame_ind+focal_frame]:
        image = cv2.imread(file)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    images = np.stack(images)

    background_sum = np.sum(images[:,:,:,2], 0, dtype=np.int16)

    bat_thresh = .1
    bat_area = 1

    total_frames = len(frame_files)

    bat_centers, bat_areas, bat_contours, rect_angles, bat_sizes, bat_rects, bat_thresh, binary = process_frame(
        images, focal_frame_ind, bat_thresh, background_sum, bat_area, debug=True)

    mask = np.zeros_like(binary, dtype=np.uint8)
    mask = cv2.drawContours(mask, bat_contours, -1, 255, -1)

    if save_images:
        shutil.copy(frame_files[focal_frame], os.path.join(out_folder, 'images'))
        mask_name = os.path.splitext(os.path.basename(frame_files[focal_frame]))[0]
        mask_file = os.path.join(out_folder, 'masks', mask_name + '.png')
        cv2.imwrite(mask_file, mask)
        
    if return_result:
        return images[focal_frame_ind], mask
    
    
def generate_annotation_frame_with_tracks(info_dict):
    """ Generate mask for image and save both.
    
    Args:
        info_dict: dictionary that contains:
            frame_files: list of frame files in clip
            focal_frame: frame ind to process
            save_folder: Path to folder to save images and masks
            save_images: boolean should image and mask be saved
        
    """
    frame_files = info_dict['frame_files']
    focal_frame_ind = info_dict['focal_frame']
    save_folder = info_dict['save_folder']
    save_images = info_dict['save_images']
    return_result = info_dict['return_result']
    min_track_length = info_dict['min_track_length']
    
    if info_dict['annotation_num'] % 100 == 0:
        print('annotation number {} started.'.format(info_dict['annotation_num']))
    
    bat_thresh = .1
    bat_area_thresh = 1
 
    track_list, positions = kbf.get_tracked_bats_in_frame(
        frame_files, focal_frame_ind=focal_frame_ind, bat_thresh=bat_thresh, 
        bat_area_thresh=bat_area_thresh
    )
    ind = 0
    
    image = plt.imread(frame_files[focal_frame_ind + ind])
    mask = np.zeros_like(image, dtype=np.uint8)
    
    num_tracks_drawn = 0
    num_short_tracks = 0
    
    for track in track_list:
        if (track['first_frame'] <= ind and len(track['track']) >= min_track_length):
            cnt = track['contour'][ind]
            if not np.any(np.isnan(cnt)):
                if len(track['track']) == 2:
                    color = [255, 255, 255]
                else:
                    color = [255, 255, 255]
                cv2.drawContours(mask, [cnt], 0, color, -1)
                num_tracks_drawn += 1
        if len(track['track']) == 2:
            num_short_tracks += 1
#     print('num tracks drawn: {} num short: {}'.format(num_tracks_drawn, num_short_tracks))            

    if save_images:
        shutil.copy(frame_files[focal_frame_ind], os.path.join(save_folder, 'images'))
        mask_name = os.path.splitext(os.path.basename(frame_files[focal_frame_ind]))[0]
        mask_file = os.path.join(save_folder, 'masks', mask_name + '.png')
        cv2.imwrite(mask_file, mask)
        
    if return_result:
        return image, mask
    else:
        return None, None

In [ ]:
# image_folder = '.../bats/18Nov/BBC/GH029844'
image_folder_root = ".../Elements/bats/17Nov"

camera_files = sorted(glob.glob(os.path.join(image_folder_root, '*')))
camera_names = [os.path.basename(file) for file in camera_files]
print(camera_names)

In [ ]:
out_folder = '.../kasanka-bats/annotations/test'
os.makedirs(out_folder, exist_ok=True)
os.makedirs(os.path.join(out_folder, 'masks'), exist_ok=True)
os.makedirs(os.path.join(out_folder, 'images'), exist_ok=True)

image_folder_root = ".../Elements/bats/17Nov"

In [ ]:
annotation_info_dict = {}

annotation_info_dict['17'] = {
    'BBC': {"first_frame": 5000, "last_frame": 15000, "min_track_length": 3},
    'Chyniangale': {"first_frame": 15001, "last_frame": 26000, "min_track_length": 3},
    'FibweParking2': {"first_frame": 19000, "last_frame": 34000, "min_track_length": 3},
    'Fibwe_Public': {"first_frame": 25000, "last_frame": 35800, "min_track_length": 3},
    'MusoleTower': {"first_frame": 28500, "last_frame": 45000, "min_track_length": 2},
#     'Musole_Parking': {"first_frame": 30000, "last_frame": 40000, "min_track_length": 3},
    'MusoleParking': {"first_frame": 40000, "last_frame": 60000, "min_track_length": 1},
    'Musole_Path2': {"first_frame": 5000, "last_frame": 13000, "min_track_length": 3},
    'NotChyniangale': {"first_frame": 3000, "last_frame": 15000, "min_track_length": 3},
    'Puku': {"first_frame": 5500, "last_frame": 11500, "min_track_length": 3},
    'Sunset': {"first_frame": None, "last_frame": None, "min_track_length": 3},
}

# annotation_info_dict['18'] = {'BBC': {"first_frame": None, "last_frame": None},
#                               'Chyniangale': {"first_frame": None, "last_frame": None},
#                               'FibweParking2': {"first_frame": 15000, "last_frame": 27000},
#                               'Fibwe_Public': {"first_frame": 22000, "last_frame": 42000},
#                               'MusoleTower': {"first_frame": 25000, "last_frame": 32000},
#                               'Musole_Parking': {"first_frame": 30000, "last_frame": 40000},
#                               'Musole_Path2': {"first_frame": 3000, "last_frame": 18000},
#                               'NotChyniangale': {"first_frame": 2000, "last_frame": 12000},
#                               'Puku': {"first_frame": None, "last_frame": None},
#                               'Sunset': {"first_frame": None, "last_frame": None},
#                              }

In [ ]:
camera_ind = 9

camera_name = list(annotation_info_dict['17'].keys())[camera_ind]

camera_name = "MusoleParking"

frame_files = sorted(glob.glob(os.path.join(image_folder_root, camera_name, '*/*.jpg')))
print(len(frame_files))
print(frame_files[0])

In [ ]:
out_folder

In [ ]:
# View outputs

random_inds = False # if false then pick evenly spaced
num_frames = 5

# camera_ind = 0
# camera_name = list(annotation_info_dict['17'].keys())[camera_ind]

info_dicts = []

annotation_num = 0

camera_name = 'MusoleParking'
save_images = False
show_images = True



frame_files = sorted(glob.glob(os.path.join(image_folder_root, camera_name, '*/*.jpg')))

print('There are {} frames'.format(len(frame_files)))
print(frame_files[0])

first_frame = annotation_info_dict['17'][camera_name]['first_frame']
last_frame = annotation_info_dict['17'][camera_name]['last_frame']
min_track_length = annotation_info_dict['17'][camera_name]['min_track_length']



if (first_frame is not None) and (last_frame is not None):
    if random_inds:
        focal_frame_inds = np.random.randint(first_frame, last_frame, num_frames)
    else:
        focal_frame_inds = np.linspace(first_frame, last_frame, num_frames, dtype=int)


    for focal_frame_ind in focal_frame_inds:

        info_dict = {'frame_files': frame_files,
                     'focal_frame': focal_frame_ind,
                     'save_folder': out_folder,
                     'save_images': save_images,
                     'return_result': show_images,
                     'min_track_length': min_track_length, 
                     'annotation_num': annotation_num
                    }
        info_dicts.append(info_dict)
        annotation_num += 1
print('{} dictionaries created'.format(len(info_dicts)))  

for info_dict in info_dicts:
    im, mask = generate_annotation_frame_with_tracks(info_dict)
    plt.figure(figsize=(20, 20))
    plt.imshow(im)
    plt.figure(figsize=(20, 20))
    
    scaled_im = np.copy(im)
    scaled_im -= np.min(scaled_im)
    scaled_im = scaled_im / np.max(scaled_im)
    plt.imshow(scaled_im)
    plt.figure(figsize=(20, 20))
    plt.imshow(mask)

In [ ]:
random_inds = True # if false then pick evenly spaced
num_frames = 100

# camera_ind = 0
# camera_name = list(annotation_info_dict['17'].keys())[camera_ind]

info_dicts = []

annotation_num = 0

for camera_name in annotation_info_dict['17']:

    frame_files = sorted(glob.glob(os.path.join(image_folder_root, camera_name, '*/*.jpg')))

    print('There are {} frames'.format(len(frame_files)))
    print(frame_files[0])

    first_frame = annotation_info_dict['17'][camera_name]['first_frame']
    last_frame = annotation_info_dict['17'][camera_name]['last_frame']
    min_track_length = annotation_info_dict['17'][camera_name]['min_track_length']

    
    
    if (first_frame is not None) and (last_frame is not None):
        if random_inds:
            focal_frame_inds = np.random.randint(first_frame, last_frame, num_frames)
        else:
            focal_frame_inds = np.linspace(first_frame, last_frame, num_frames, dtype=int)
                                      

        for focal_frame_ind in focal_frame_inds:
            save_images = True
            show_images = False
            info_dict = {'frame_files': frame_files,
                         'focal_frame': focal_frame_ind,
                         'save_folder': out_folder,
                         'save_images': save_images,
                         'return_result': show_images,
                         'min_track_length': min_track_length, 
                         'annotation_num': annotation_num
                        }
            info_dicts.append(info_dict)
            annotation_num += 1
print('{} dictionaries created'.format(len(info_dicts)))         

In [ ]:
for ind in range(7):
    print(info_dicts[ind]['save_folder'])

In [ ]:
from multiprocessing import Pool

if __name__ == '__main__':
    with Pool(7) as p:
        p.map(generate_annotation_frame_with_tracks, info_dicts)

In [ ]:
image, mask = generate_annotation_frame_with_tracks(info_dict)
            if show_images:
                plt.figure(figsize=(20, 20))
                plt.imshow(image)
                plt.imshow(mask, alpha=.3)
                plt.title('Frame num {}'.format(focal_frame_ind))

In [ ]:
for ind in range(5):
    print(info_dicts[ind]['focal_frame'])

In [ ]:
focal_frame_ind = 14900

# out_folder = ' '
save_images = False
show_images = True
info_dict = {'frame_files': frame_files,
             'focal_frame': focal_frame_ind,
             'save_folder': out_folder,
             'save_images': save_images,
             'return_result': show_images,
             'min_track_length': 3
            }
info_dicts[0]['return_result'] = True
image, mask = generate_annotation_frame_with_tracks(info_dicts[0])

plt.figure(figsize=(20, 20))
plt.imshow(image)
plt.imshow(mask, alpha=.2)
plt.title('Frame num {}'.format(focal_frame_ind))
plt.figure(figsize=(20, 20))
plt.imshow(mask)

In [ ]:
# info_dicts = []

# for camera_ind, camera_name in enumerate(camera_names[:]):
    
#     first_frame = annotation_info_dict['17'][camera_name]['first_frame']
#     last_frame = annotation_info_dict['17'][camera_name]['last_frame']
    
#     if first_frame is None or last_frame is None:
#         continue
        
#     print(camera_name)
    
#     frame_files = sorted(glob.glob(os.path.join(camera_files[camera_ind], '*/*.jpg')))
#     print(frame_files[0])

#     frame_inds = np.linspace(first_frame, last_frame, num_images_per_camera, dtype=int)

#     for num_frames, focal_ind in enumerate(frame_inds):

#         info_dict = {'frame_files': frame_files,
#                      'focal_frame': focal_ind,
#                      'save_folder': out_folder,
#                      'save_images': save_images,
#                      'return_result': show_images
#                     }

#         info_dicts.append(info_dict)


#         if show_images:
#             image, mask = generate_annotation_frame(info_dict)
#             plt.figure(figsize=(10, 10))
#             plt.imshow(image)
#             plt.figure(figsize=(10, 10))
#             plt.imshow(mask)
        
# print('{} images are ready to be processed. Saving is set to {}'.format(len(info_dicts), save_images))

In [ ]:
# image_files = glob.glob(image_folder + '/*.jpg')
# image_files.sort()
# print('There are {} frames'.format(len(frame_files)))


# bat_thresh = .1
# bat_area_thresh = 1
# focal_frame_ind = 23999
# track_list, positions = kbf.get_tracked_bats_in_frame(frame_files, focal_frame_ind=focal_frame_ind, 
#                                        bat_thresh=bat_thresh, 
#                                        bat_area_thresh=bat_area_thresh
#                                        )

In [ ]:
# first_frames = [len(track['track']) for track in track_list if track['first_frame']==0]
# plt.hist(first_frames)

In [ ]:
# track_list[0]['last_frame']

In [ ]:
# for track in track_list:
#     if track['last_frame'] == 0:
#         break

In [ ]:
figure_scale=60

In [ ]:
# image = plt.imread(frame_files[focal_frame_ind])
# plt.figure(figsize = (int(image.shape[1] / figure_scale), int(image.shape[0] / figure_scale)))
# plt.imshow(image)
# plt.scatter(positions[0][:,0], positions[0][:,1], c='red', alpha=.5)

In [ ]:
x = np.array([1, 1])
print(bool(np.squeeze(x).shape))

In [ ]:
# focal_frame_ind = 8333

out_folder = ' '
save_images = False
show_images = True
info_dict = {'frame_files': frame_files,
             'focal_frame': focal_frame_ind,
             'save_folder': out_folder,
             'save_images': save_images,
             'return_result': show_images
            }
image, mask = generate_annotation_frame_with_tracks(info_dict)

plt.figure(figsize=(20, 20))
plt.imshow(image)
# plt.figure(figsize=(20, 20))
plt.imshow(mask, alpha=.5)

In [ ]:
bat_thresh = .1
bat_area_thresh = 1
# focal_frame_ind = 15000
track_list, positions, distances, max_distances, active_list, all_pre_distances, all_row_inds, all_col_inds = kbf.get_tracked_bats_in_frame(frame_files, focal_frame_ind=focal_frame_ind, 
                                       bat_thresh=bat_thresh, 
                                       bat_area_thresh=bat_area_thresh
                                       )



In [ ]:
np.min(distances[0][76])

In [ ]:
# print(np.argwhere(distances[0]==9.668213951199597))
# print(np.argwhere(distances[0]==17.55639684181554))
# print(distances[0][75, 37])
# print(np.min(distances[0][:, 23]))

# print(np.argwhere(all_pre_distances[0]==9.280355596635292))
# print(all_pre_distances[0][75, 63])
# print(np.min(all_pre_distances[0][:, 37]))

# print(distances[0].shape, all_pre_distances[0].shape)

In [ ]:
for pos_ind, position in enumerate(positions[1]):
    if (position[0] > 700) and (position[0] < 1000):
        if (position[1] > 500) and (position[1] < 700):
            print(pos_ind)
            view_inds.append(track_ind)
    

In [ ]:
view_inds = []

for track_ind, track in enumerate(track_list):
    if (track['track'][0, 0] > 900) and (track['track'][0, 0] < 1100):
        if (track['track'][0, 1] > 900) and (track['track'][0, 1] < 1100):
            print(track_ind)
            view_inds.append(track_ind)

In [ ]:
from scipy.optimize import linear_sum_assignment
r, c = linear_sum_assignment(distances[0])

In [ ]:
ind = 75
print(r[ind])
print(np.min(distances[0][r[ind]]))
print(np.argwhere(c==73))
distances[0][r[ind], c[ind]]

In [ ]:
print(r[78])
print(np.argmin(distances[0][r[78]]))
print(np.argwhere(c==75))
distances[0][r[78], c[78]]

In [ ]:
print(np.argmin(distances[0][r[76]]))
print(np.argwhere(c==74))
distances[0][r[76], c[76]]


In [ ]:

first_frame = 0
focal_frame = 0 + first_frame
figure_scale = 60

max_ind = 1

plt_ind = 266

for focal_frame in range(0, 3):
    num_drawn = 0
    # for focal_frame in range(focal_frame - 2, focal_frame + 3, 2):
    image = plt.imread(frame_files[focal_frame+focal_frame_ind])
    images = [image]
    mask = np.zeros_like(image, dtype=np.uint8)
    plt.figure(figsize = (int(images[0].shape[1] / figure_scale), int(images[0].shape[0] / figure_scale)))
    plt.imshow(image)
    
    plt.scatter(positions[focal_frame-first_frame][:, 0], 
                positions[focal_frame-first_frame][:, 1], c='white', alpha=.5, s=50)
    
    for track_ind, track in enumerate(track_list):
        if (track['last_frame'] >= focal_frame-first_frame 
            and track['first_frame'] <= focal_frame-first_frame
           and len(track['track']) >= 1):
            if track_ind in view_inds or False:
                rel_frame = focal_frame - track['first_frame'] - first_frame
                plt.plot(track['track'][:rel_frame, 0], track['track'][:rel_frame, 1], linewidth=2)
                num_drawn += 1
                if len(track['track']) == 2:
                    color = 'red'
                elif len(track['track']) == 3:
                    color = 'yellow'
                else:
                    color = 'green'

                plt.scatter(track['track'][rel_frame, 0], track['track'][rel_frame, 1], s=25, c=color)
                cnt = track['contour'][rel_frame]
                if not np.any(np.isnan(cnt)):
                    try:
                        cv2.drawContours(mask, [cnt], 0, 255, -1)
                    except:
                        print(cnt, cnt==np.nan)

    
#     plt.scatter(positions[focal_frame-first_frame][201,0], 
#                 positions[focal_frame-first_frame][201,1], c='red', alpha=.5, s=100)
#     plt.scatter(positions[focal_frame-first_frame][:,0], 
#                 positions[focal_frame-first_frame][:,1], c='red', alpha=.5, s=20)
#     plt.scatter(positions[focal_frame-first_frame][plt_args,0], 
#             positions[focal_frame-first_frame][plt_args,1], c='red', alpha=.5, s=20)
    
#     plt.scatter(positions[focal_frame+1-first_frame][:max_ind,0], positions[focal_frame+1-first_frame][:max_ind,1], c='green', alpha=.5)
#     plt.figure(figsize = (int(images[0].shape[1] / figure_scale), int(images[0].shape[0] / figure_scale)))
    plt.imshow(mask, alpha=.3)
    print(num_drawn)

In [ ]:
track_list[16]

In [ ]:
image = plt.imread(image_files[focal_frame_ind])
plt.imshow(image)
for track in tracks:
    plt.plot()

In [ ]:
length = [len(track['size']) for track in tracks]
plt.hist(length)

In [ ]:
for track in track_list[:3]:
    plt.scatter(track['track'][:,0], track['track'][:,1])

In [ ]:
info_dict = {'frame_files': frame_files,
             'focal_frame': focal_ind,
             'save_folder': out_folder,
             'save_images': False,
             'return_result': True
            }

image, mask = generate_annotation_frame_with_tracks(info_dict)

plt.figure(figsize=(20, 20))
plt.imshow(image)
plt.figure(figsize=(20, 20))
plt.imshow(mask)

In [ ]:
focal_ind = 12000

info_dict = {'frame_files': frame_files,
             'focal_frame': focal_ind,
             'save_folder': out_folder,
             'save_images': False,
             'return_result': True
            }

image, mask = generate_annotation_frame(info_dict)
plt.figure(figsize=(20, 20))
plt.imshow(image)
plt.figure(figsize=(20, 20))
plt.imshow(mask)

In [ ]:
np.linspace(5000, 15000, 2, dtype=int)

In [ ]:
training_images = []
training_masks = []

save_images = True
show_images = False

num_images_per_camera = 200
max_folder_ind = 0
first_frame = 15000
last_frame = 27000

show_every_n_images = 20

info_dicts = []

for camera_ind, camera_name in enumerate(camera_names[:1]):
    
    first_frame = annotation_info_dict['17'][camera_name]['first_frame']
    last_frame = annotation_info_dict['17'][camera_name]['last_frame']
    
    if first_frame is None or last_frame is None:
        continue
        
    print(camera_name)
    
    frame_files = sorted(glob.glob(os.path.join(camera_files[camera_ind], '*/*.jpg')))
    print(frame_files[0])

    frame_inds = np.linspace(first_frame, last_frame, num_images_per_camera, dtype=int)

    for num_frames, focal_ind in enumerate(frame_inds):

        info_dict = {'frame_files': frame_files,
                     'focal_frame': focal_ind,
                     'save_folder': out_folder,
                     'save_images': save_images,
                     'return_result': show_images
                    }

        info_dicts.append(info_dict)


        if show_images:
            image, mask = generate_annotation_frame(info_dict)
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.figure(figsize=(10, 10))
            plt.imshow(mask)
        
print('{} images are ready to be processed. Saving is set to {}'.format(len(info_dicts), save_images))

In [ ]:
from multiprocessing import Pool

if __name__ == '__main__':
    with Pool(7) as p:
        p.map(generate_annotation_frame, info_dicts)

In [ ]:
def compute_mean(images):
    """Computer mean of each color channel from a list of images.
    
    Args:
        images: list of 3D numpy array HWC
    Returns:
        [mean red, mean green, mean blue]
    """
    color_sums = np.zeros(3)
    total_pixels = 0
    for image in images:
        color_sums += np.sum(np.sum(image, 0), 0)
        total_pixels += image.shape[0] * image.shape[1]
    return color_sums / total_pixels
        
def compute_std(images):
    """Computer standard deviation of each color channel from a list of images.
    
    Args:
        images: list of 3D numpy array HWC
    Returns:
        [std red, std green, std blue]
    """
    
    color_dif_sums = np.zeros(3)
    total_pixels = 0
    for image in images:
        color_means = np.mean(np.mean(image, 0), 0)
        dif = image - color_means
        dif2 = dif ** 2
        color_dif_sums += np.sum(np.sum(dif2, 0), 0)
        total_pixels += image.shape[0] * image.shape[1]
    std = np.sqrt(color_dif_sums / (total_pixels-1))
    return std

In [ ]:
os.path.join(out_folder, 'data/mean.npy')

In [ ]:
mean = compute_mean(training_images)
std = compute_std(training_images)

print('mean', mean, 'std', std)



In [ ]:
np.save(os.path.join(out_folder, 'mean.npy'), mean[::-1])
np.save(os.path.join(out_folder, 'std.npy'), std[::-1])

In [ ]:
image_files  = sorted(glob.glob(os.path.join(out_folder, 'images/*.jpg')))

In [ ]:
training_images = [cv2.imread(file) for file in image_files]

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(training_images[-1])
plt.figure(figsize=(20,20))
plt.imshow(training_masks[-1])